## Setup and Data Import

In [7]:
import sys
sys.path.insert(0, '..')

from joblib import load

import Sita_Functions as fxns
from Sita_Functions import np, pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import timedelta

## Claims DF

In [ ]:
!python ../Preprocessing.py

In [3]:
claims = load('claims.pkl')

### New Columns

In [ ]:
claims['AgeAtService'] = ((claims.ClaimStartDt - claims.DOB)
                              / timedelta(days=365)).astype(int)
claims['HasDied'] = claims.DOD.notna()

physician_cols = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
claims['HasAnyPhysician'] = claims[physician_cols].notna().any(axis=1)
claims['HasAllPhysicians'] = claims[physician_cols].notna().all(axis=1)

claims['ClaimDuration'] = \
    (claims.ClaimEndDt  - claims.ClaimStartDt).dt.days + 1
claims['IPDuration'] = \
    (claims.DischargeDt - claims.AdmissionDt).dt.days + 1

claims['ClaimCost'] = \
    claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid
claims['PercInsCovered'] = \
    round((claims.InscClaimAmtReimbursed / claims.ClaimCost) * 100)
claims['DailyClaimCost'] = \
    round(claims.ClaimCost / claims.ClaimDuration)

# add date cols containing only day, week, year
fxns.split_date(claims, ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt'])

# claims per patient
# ratio inpatient claims

### Pre-processing

In [ ]:
# encode bool colums to 0/1
fxns.re_encode_bool(claims, ['HasDied', 'HasAnyPhysician', 'HasAllPhysicians'])

In [ ]:
# drop ChronicCond_ prefix from applicable cols
fxns.drop_chronic_prefix(claims)

In [ ]:
# change object cols to dtype category
fxns.to_category_dtype(claims)

### Variables

In [ ]:
numeric_cols     = fxns.cols_by_dtype(claims)[0]
categorical_cols = fxns.cols_by_dtype(claims)[1]
date_cols        = fxns.cols_by_dtype(claims)[2]

physician_cols = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
chronic_cols   = \
    claims.columns[claims.columns.str.contains('Chronic')].to_list()
diagnosis_cols = \
    claims.columns[claims.columns.str.contains('Diagnosis')].to_list()
procedure_cols = \
    claims.columns[claims.columns.str.contains('Procedure')].to_list()

### Summary Data

In [ ]:
outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]

In [ ]:
groupby_providers   = claims.groupby('Provider')

In [ ]:
mean_by_provider    = \
    groupby_providers.mean().add_suffix('_mean').reset_index()

In [ ]:
sum_by_provider     = \
    groupby_providers.sum().add_suffix('_sum').reset_index()

In [ ]:
chronic_by_provider = claims[['Provider'] + chronic_cols
                            ].groupby('Provider').sum().reset_index()

In [ ]:
# need to get unique counts
count_by_provider = groupby_providers[
    ['BeneID', 'ClaimID', 'County', 'State', 'IPDuration']
    + physician_cols + diagnosis_cols + procedure_cols
    ].count().add_suffix('_count').reset_index()

# ADD PHYSICIAN COLS (NONE/MULT), RATIOS, SUMS OF EACH
# CLAIM ADMIT CODE, DIAGNOSIS GROUP CODE?
# ADD RANGE OF AGE

In [ ]:
providers = pd.merge(mean_by_provider, sum_by_provider, on='Provider')
providers = pd.merge(providers, count_by_provider, on='Provider')
providers = pd.merge(providers, chronic_by_provider, on='Provider')
providers.sort_index(axis=1, inplace=True)

In [ ]:
# print(claims.columns, '\n')

# Can't use, but have IPDuration count/mean/sum:
#     'AdmissionDt', 'AdmissionDt_Week',
#     'DischargeDt', 'DischargeDt_Week'

# Can't use, but have ClaimDuration count/mean/sum:
#     'ClaimEndDt', 'ClaimEndDt_Week', 'ClaimID'
#     'ClaimStartDt', 'ClaimStartDt_Week',


# Can't use, but have AgeAtService mean/sum and HasDied sum/ratio:
#     'DOB', 'DOD'

In [ ]:
print(providers.shape, '\n')
print(providers.columns)

## New Columns

### Beneficiaries

In [ ]:
def binary_ratios(df, col, group1, group2):
    by_provider = df.groupby(['Provider', col]).ClaimID.count().reset_index()

    colname = pd.DataFrame()
    colname[group1] = by_provider[by_provider[col] == 1].ClaimID.to_list()
    colname[group2] = by_provider[by_provider[col] == 0].ClaimID.to_list()
    
    newcolname1 = f'{group1}_ratio'
    
    providers[newcolname1] = round((colname[group1] / (colname[group1] + colname[group2])) * 100, 2)

binary_ratios(claims, 'Gender', 'GenderZero', 'GenderOne')
binary_ratios(claims, 'IsOutpatient', 'IsOutpatient', 'Inpatient')
binary_ratios(claims, 'RenalDisease', 'HasRenalDisease', 'NotRenalDisease')
binary_ratios(claims, 'HasDied', 'HasDied', 'NotDead')

In [ ]:
race_by_provider = claims.groupby(['Provider', 'Race']).ClaimID.count().reset_index()

race = pd.DataFrame()
race['race1'] = race_by_provider[race_by_provider.Race == 1].ClaimID.to_list()
race['race0'] = race_by_provider[race_by_provider.Race == 0].ClaimID.to_list()
race['race3'] = race_by_provider[race_by_provider.Race == 3].ClaimID.to_list()
race['race5'] = race_by_provider[race_by_provider.Race == 5].ClaimID.to_list()

providers['RaceZero_ratio']   = round((race.race0 / np.sum(race, axis=1) * 100), 2)
providers['RaceThree_ratio'] = round((race.race3 / np.sum(race, axis=1) * 100), 2)
providers['RaceFive_ratio']  = round((race.race5 / np.sum(race, axis=1) * 100), 2)

In [ ]:
# DO WE WANT CHRONIC CONDITION RATIOS? WHY/WHY NOT?

In [ ]:
ratio_cols = providers.columns[providers.columns.str.contains('_ratio')].to_list()
providers[['Provider'] + ratio_cols].sample(5)

### Doctors

### Codes

### Money

### Time

### Location

### Size